In [1]:
import bittensor as bt
import random
import os
import sys
import numpy as np
import pandas as pd
import hashlib
from collections import defaultdict
from Crypto import Random
from Crypto.Random import random
from Crypto.PublicKey import ECC
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from pprint import pprint, pformat
import binascii


In [2]:
class ECCommitment:
    def __init__(self, g, h):
        self.g = g  # Base point of the curve
        self.h = h  # Another random point on the curve

    def commit(self, m):  # AKA Seal.
        m_val = hash_data(m)  # Compute hash of the data
        r = random.randint(1, 2**256)
        c1 = self.g.__mul__(m_val)
        c2 = self.h.__mul__(r)
        c = c1.__add__(c2)
        print(
            f"Committing: Data = {m}\nHashed Value = {m_val}\nRandom Value = {r}\nComputed Commitment = {c}\n"
        )
        return c, m_val, r

    def open(self, c, m_val, r):
        c1 = self.g.__mul__(m_val)
        c2 = self.h.__mul__(r)
        computed_c = c1.__add__(c2)
        print(
            f"\nOpening: Hashed Value = {m_val}\nRandom Value = {r}\nRecomputed Commitment = {computed_c}\nOriginal Commitment = {c}"
        )
        return computed_c == c


def setup_CRS():
    curve = ECC.generate(curve="P-256")
    g = curve.pointQ  # Base point
    h = ECC.generate(curve="P-256").pointQ  # Another random point
    return g, h


In [3]:
g, h = setup_CRS()
print("g:", g)
print("h:", h)

g: <Crypto.PublicKey.ECC.EccPoint object at 0x7f6cf20c8e80>
h: <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b81521f30>


In [4]:
def make_random_file(name=None, maxsize=1024):
    size = random.randint(32, maxsize)
    data = os.urandom(size)
    if isinstance(name, str):
        with open(name, "wb") as fout:
            fout.write(data)
    else:
        return data

f = random_file = make_random_file()
len(f)

576

In [5]:
def encrypt_data(filename, key):
    """
    Encrypt the data in the given filename using AES-GCM.

    Parameters:
    - filename: str or bytes. If str, it's considered as a file name. If bytes, as the data itself.
    - key: bytes. 16-byte (128-bit), 24-byte (192-bit), or 32-byte (256-bit) secret key.

    Returns:
    - cipher_text: bytes. The encrypted data.
    - nonce: bytes. The nonce used for the GCM mode.
    - tag: bytes. The tag for authentication.
    """

    # If filename is a string, treat it as a file name and read the data
    if isinstance(filename, str):
        with open(filename, "rb") as file:
            data = file.read()
    else:
        data = filename

    # Initialize AES-GCM cipher
    cipher = AES.new(key, AES.MODE_GCM)
    
    # Encrypt the data
    cipher_text, tag = cipher.encrypt_and_digest(data)

    return cipher_text, cipher.nonce, tag

# Example usage
key = get_random_bytes(32)  # 256-bit key
encrypted_data, nonce, tag = encrypt_data(random_file, key)
encrypted_data

b'\xa8\'I\x7f\xedM~\xd5\xf8\xef\x960\xb1#\xbd}}\xcdvQ4\xbd2{\x95\tb\xcb\x155\xf1;j3\xe1\xbd\x9d\x08$:\x84Ea\x18\x8a\xb8~\xd0VN\xccn\xac\x11}\x03\xf7\xc2\x89;+\xedD\xb4E\xc0S\xa5t\x0fL\x81*\xdc\xf8eX\xd7\x11[\xfc>P_P\xfe\xe1\xd0\x1f\xa1\x14j\xf4+\xb9\xf0#p0\x89\x7f\xd0\xbb\x0c\x99J^\xc7\n\xb4\x0e\xc4q\x85\xe8\x1f\xd8\x98\xf3\xb8\xc4\x9a\x18\'\xae\x9c\x04\x85\xb8\x9d\xa5<\x0f\x8e$R\xd1\xc5\x86\xb38\x90\xc3\x83\x9br\x1a\x8e\xb0\xf1\xb7=u\xe8N\x80XU\x13zy03\xdf\xfeXh\xb9\xef\xdbq9=>\x00\x9ao\xbf\xbc\xb9L\x14\xd3\xd4K&~\xa6\xb1\xb4\xb1\x82\xb7;U{T\xe2\x12\xad\xfe\xa4+\xc2m\xd8P\xe4\x04\xe6~\xc0\x9a\x1c\x11i\x05sN\x0c3\xa3Rn2\x80\x03\x079\x07a[\xd0\xf6\xfe\xa0J>HP\x8eZL\xaf\xae\x17\xce\xee\xa1\x1f\xd7?\xe1M\xa1\xef\xd1\x1fp8\x9e\xbba\x132\xca\x0fS\xb4\xed\xaf-\xeb\xdf?!\xde\x9d\xb3\x8d\xc4\x80aW\x8d|o\xccw\xe53\x97:\xe6\xf10\xd9\xc8\x1e\xb9\xbd\x7fs\xaa!{\xa1/\x13\x0eAC\xbc\xb5B\xc0\xa1G\xc5\xe2\xcf\xdd\xeb\xc8\xaf\xc0\xd6c\x18\xf5\xe8\x15!\xb7\x90""\xef\xd8\xda\xbfy\\JB\xdb>\xddT\x13\x0f1s\

In [6]:
# Hash encrypted data E with SHA3-256 (args: encrypted data E) to get hash H
def hash_data(data):
    if not isinstance(data, (bytes, bytearray)):
        data_str = str(data)
        data = data_str.encode()
    h = hashlib.sha3_256(data).hexdigest()
    return int(h, 16)

H = hash_data(encrypted_data)
H

11571670828956623239669574103713213937807564828886038512947311940781227344988

In [7]:
# Determine a random chunksize between 2kb-128kb (random sample from this range) store as chunksize_E
def get_random_chunksize(maxsize=128):
    return random.randint(2, maxsize)

bytesize = sys.getsizeof(encrypted_data)
print("bytesize:", bytesize)
chunksize = get_random_chunksize(bytesize // 4)
print("chunksize:", chunksize)

bytesize: 609
chunksize: 90


In [8]:
def chunk_data(data, chunksize: int):
    for i in range(0, len(data), chunksize):
        yield data[i : i + chunksize]

chunks = list(chunk_data(encrypted_data, chunksize))
print("n chunks:", len(chunks))
pprint(f"chunks: {chunks}")

n chunks: 7
('chunks: '
 '[b"\\xa8\'I\\x7f\\xedM~\\xd5\\xf8\\xef\\x960\\xb1#\\xbd}}\\xcdvQ4\\xbd2{\\x95\\tb\\xcb\\x155\\xf1;j3\\xe1\\xbd\\x9d\\x08$:\\x84Ea\\x18\\x8a\\xb8~\\xd0VN\\xccn\\xac\\x11}\\x03\\xf7\\xc2\\x89;+\\xedD\\xb4E\\xc0S\\xa5t\\x0fL\\x81*\\xdc\\xf8eX\\xd7\\x11[\\xfc>P_P\\xfe\\xe1\\xd0\\x1f\\xa1", '
 'b"\\x14j\\xf4+\\xb9\\xf0#p0\\x89\\x7f\\xd0\\xbb\\x0c\\x99J^\\xc7\\n\\xb4\\x0e\\xc4q\\x85\\xe8\\x1f\\xd8\\x98\\xf3\\xb8\\xc4\\x9a\\x18\'\\xae\\x9c\\x04\\x85\\xb8\\x9d\\xa5<\\x0f\\x8e$R\\xd1\\xc5\\x86\\xb38\\x90\\xc3\\x83\\x9br\\x1a\\x8e\\xb0\\xf1\\xb7=u\\xe8N\\x80XU\\x13zy03\\xdf\\xfeXh\\xb9\\xef\\xdbq9=>\\x00\\x9ao\\xbf\\xbc\\xb9", '
 "b'L\\x14\\xd3\\xd4K&~\\xa6\\xb1\\xb4\\xb1\\x82\\xb7;U{T\\xe2\\x12\\xad\\xfe\\xa4+\\xc2m\\xd8P\\xe4\\x04\\xe6~\\xc0\\x9a\\x1c\\x11i\\x05sN\\x0c3\\xa3Rn2\\x80\\x03\\x079\\x07a[\\xd0\\xf6\\xfe\\xa0J>HP\\x8eZL\\xaf\\xae\\x17\\xce\\xee\\xa1\\x1f\\xd7?\\xe1M\\xa1\\xef\\xd1\\x1fp8\\x9e\\xbba\\x132\\xca\\x0fS\\xb4\\xed', "
 'b\'\\xaf-\\xeb\\xdf?!\\xde

In [9]:
committer = ECCommitment(g, h)
committer

In [10]:
class MerkleTree(object):
    def __init__(self, hash_type="sha3_256"):
        hash_type = hash_type.lower()
        if hash_type in [
            "sha256",
            "sha224",
            "sha384",
            "sha512",
            "sha3_256",
            "sha3_224",
            "sha3_384",
            "sha3_512",
        ]:
            self.hash_function = getattr(hashlib, hash_type)
        else:
            raise Exception("`hash_type` {} nor supported".format(hash_type))

        self.reset_tree()

    def _to_hex(self, x):
        try:  # python3
            return x.hex()
        except:  # python2
            return binascii.hexlify(x)

    def reset_tree(self):
        self.leaves = list()
        self.levels = None
        self.is_ready = False

    def add_leaf(self, values, do_hash=False):
        self.is_ready = False
        # check if single leaf
        if not isinstance(values, tuple) and not isinstance(values, list):
            values = [values]
        for v in values:
            if do_hash:
                v = v.encode("utf-8")
                v = self.hash_function(v).hexdigest()
            v = bytearray.fromhex(v)
            self.leaves.append(v)

    def get_leaf(self, index):
        return self._to_hex(self.leaves[index])

    def get_leaf_count(self):
        return len(self.leaves)

    def get_tree_ready_state(self):
        return self.is_ready

    def _calculate_next_level(self):
        solo_leave = None
        N = len(self.levels[0])  # number of leaves on the level
        if N % 2 == 1:  # if odd number of leaves on the level
            solo_leave = self.levels[0][-1]
            N -= 1

        new_level = []
        for l, r in zip(self.levels[0][0:N:2], self.levels[0][1:N:2]):
            new_level.append(self.hash_function(l + r).digest())
        if solo_leave is not None:
            new_level.append(solo_leave)
        self.levels = [
            new_level,
        ] + self.levels  # prepend new level

    def make_tree(self):
        self.is_ready = False
        if self.get_leaf_count() > 0:
            self.levels = [
                self.leaves,
            ]
            while len(self.levels[0]) > 1:
                self._calculate_next_level()
        self.is_ready = True

    def get_merkle_root(self):
        if self.is_ready:
            if self.levels is not None:
                return self._to_hex(self.levels[0][0])
            else:
                return None
        else:
            return None

    def get_proof(self, index):
        if self.levels is None:
            return None
        elif not self.is_ready or index > len(self.leaves) - 1 or index < 0:
            return None
        else:
            proof = []
            for x in range(len(self.levels) - 1, 0, -1):
                level_len = len(self.levels[x])
                if (index == level_len - 1) and (
                    level_len % 2 == 1
                ):  # skip if this is an odd end node
                    index = int(index / 2.0)
                    continue
                is_right_node = index % 2
                sibling_index = index - 1 if is_right_node else index + 1
                sibling_pos = "left" if is_right_node else "right"
                sibling_value = self._to_hex(self.levels[x][sibling_index])
                proof.append({sibling_pos: sibling_value})
                index = int(index / 2.0)
            return proof


    def update_leaf(self, index, new_value):
        """Update a specific leaf in the tree and propagate changes upwards."""
        if not self.is_ready:
            return None
        new_value = bytearray.fromhex(new_value)
        self.levels[-1][index] = new_value
        for x in range(len(self.levels) - 1, 0, -1):
            parent_index = index // 2
            left_child = self.levels[x][parent_index * 2]
            try:
                right_child = self.levels[x][parent_index * 2 + 1]
            except IndexError:
                right_child = bytearray()
            self.levels[x-1][parent_index] = self.hash_function(left_child + right_child).digest()
            index = parent_index

def validate_merkle_proof(proof, target_hash, merkle_root):
    merkle_root = bytearray.fromhex(merkle_root)
    target_hash = bytearray.fromhex(target_hash)
    if len(proof) == 0:
        return target_hash == merkle_root
    else:
        proof_hash = target_hash
        for p in proof:
            try:
                # the sibling is a left node
                sibling = bytearray.fromhex(p["left"])
                proof_hash = hashlib.sha3_256(sibling + proof_hash).digest()
            except:
                # the sibling is a right node
                sibling = bytearray.fromhex(p["right"])
                proof_hash = hashlib.sha3_256(proof_hash + sibling).digest()
        return proof_hash == merkle_root


In [11]:
def commit_data(committer, data_chunks):
    merkle_tree = MerkleTree()
    commitments = defaultdict(lambda: [None] * len(data_chunks))
    # commitments = {}

    for index, chunk in enumerate(data_chunks):
        c, m_val, r = committer.commit(chunk)
        commitments[index] = {
            "index": index,
            "hash": m_val,
            "data_chunk": chunk,
            "point": c,
            "randomness": r,
            "merkle_proof": None,
        }
        merkle_tree.add_leaf(ecc_point_to_hex(c))

    merkle_tree.make_tree()
    return {merkle_tree.get_merkle_root(): {'commitments': commitments, 'merkle_tree': merkle_tree}}

def ecc_point_to_hex(point):
    point_str = "{},{}".format(point.x, point.y)
    return binascii.hexlify(point_str.encode()).decode()

def hex_to_ecc_point(hex_str, curve):
    point_str = binascii.unhexlify(hex_str).decode()
    x, y = map(int, point_str.split(","))
    return ECC.EccPoint(curve, x, y)

store_data = commit_data
commitments_E = store_data(committer, chunks)
commitments_E
# TODO: once this is generaetd, we send it back to the Validator minus the random values
# {merkle_root: (data_chunks, hashes, commitment_points)} 

Committing: Data = b"\xa8'I\x7f\xedM~\xd5\xf8\xef\x960\xb1#\xbd}}\xcdvQ4\xbd2{\x95\tb\xcb\x155\xf1;j3\xe1\xbd\x9d\x08$:\x84Ea\x18\x8a\xb8~\xd0VN\xccn\xac\x11}\x03\xf7\xc2\x89;+\xedD\xb4E\xc0S\xa5t\x0fL\x81*\xdc\xf8eX\xd7\x11[\xfc>P_P\xfe\xe1\xd0\x1f\xa1"
Hashed Value = 38950804539089487672099356560253669889976772888895228717164058477895961628876
Random Value = 11371969935780734790947111972165227855775441224093653169512382489301334361355
Computed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b81211e70>

Committing: Data = b"\x14j\xf4+\xb9\xf0#p0\x89\x7f\xd0\xbb\x0c\x99J^\xc7\n\xb4\x0e\xc4q\x85\xe8\x1f\xd8\x98\xf3\xb8\xc4\x9a\x18'\xae\x9c\x04\x85\xb8\x9d\xa5<\x0f\x8e$R\xd1\xc5\x86\xb38\x90\xc3\x83\x9br\x1a\x8e\xb0\xf1\xb7=u\xe8N\x80XU\x13zy03\xdf\xfeXh\xb9\xef\xdbq9=>\x00\x9ao\xbf\xbc\xb9"
Hashed Value = 100988728033662964795533657568783397964785622474047693607566739886388900398160
Random Value = 81550652198281941712483559203026265366378569172166804112827222004288662073383
C

{'b15f3300df9da493f499823cb451ba212de07cafbb6414540ddaa801f0e5ccf3': {'commitments': defaultdict(<function __main__.commit_data.<locals>.<lambda>()>,
              {0: {'index': 0,
                'hash': 38950804539089487672099356560253669889976772888895228717164058477895961628876,
                'data_chunk': b"\xa8'I\x7f\xedM~\xd5\xf8\xef\x960\xb1#\xbd}}\xcdvQ4\xbd2{\x95\tb\xcb\x155\xf1;j3\xe1\xbd\x9d\x08$:\x84Ea\x18\x8a\xb8~\xd0VN\xccn\xac\x11}\x03\xf7\xc2\x89;+\xedD\xb4E\xc0S\xa5t\x0fL\x81*\xdc\xf8eX\xd7\x11[\xfc>P_P\xfe\xe1\xd0\x1f\xa1",
                'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f6b81211e70>,
                'randomness': 11371969935780734790947111972165227855775441224093653169512382489301334361355,
                'merkle_proof': None},
               1: {'index': 1,
                'hash': 100988728033662964795533657568783397964785622474047693607566739886388900398160,
                'data_chunk': b"\x14j\xf4+\xb9\xf0#p0\x89\x7f\xd0\xbb\x0c\x99J^\xc7\n\xb4\

In [12]:
keys = list(commitments_E.keys())
keys

['b15f3300df9da493f499823cb451ba212de07cafbb6414540ddaa801f0e5ccf3']

In [13]:
inner_dict = commitments_E[keys[0]]
inner_dict

{'commitments': defaultdict(<function __main__.commit_data.<locals>.<lambda>()>,
             {0: {'index': 0,
               'hash': 38950804539089487672099356560253669889976772888895228717164058477895961628876,
               'data_chunk': b"\xa8'I\x7f\xedM~\xd5\xf8\xef\x960\xb1#\xbd}}\xcdvQ4\xbd2{\x95\tb\xcb\x155\xf1;j3\xe1\xbd\x9d\x08$:\x84Ea\x18\x8a\xb8~\xd0VN\xccn\xac\x11}\x03\xf7\xc2\x89;+\xedD\xb4E\xc0S\xa5t\x0fL\x81*\xdc\xf8eX\xd7\x11[\xfc>P_P\xfe\xe1\xd0\x1f\xa1",
               'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f6b81211e70>,
               'randomness': 11371969935780734790947111972165227855775441224093653169512382489301334361355,
               'merkle_proof': None},
              1: {'index': 1,
               'hash': 100988728033662964795533657568783397964785622474047693607566739886388900398160,
               'data_chunk': b"\x14j\xf4+\xb9\xf0#p0\x89\x7f\xd0\xbb\x0c\x99J^\xc7\n\xb4\x0e\xc4q\x85\xe8\x1f\xd8\x98\xf3\xb8\xc4\x9a\x18'\xae\x9c\x04\x85\xb8\x9d\xa5<

In [14]:
hashes = inner_dict["commitments"]["hashes"]
assert len(hashes) == len(chunks)

In [15]:
def get_challenge_indices(num_chunks, factor=0.1):
    return random.sample(list(range(num_chunks)), int(1 + num_chunks * factor))

ci = challenge_indices = get_challenge_indices(len(chunks))
ci

[5]

In [16]:
def get_merkle_root_to_challenge(commitments):
    merkle_roots = list(commitments.keys())
    return random.choice(merkle_roots)

mrc = merkle_root_challenge = get_merkle_root_to_challenge(commitments_E)
mrc

'b15f3300df9da493f499823cb451ba212de07cafbb6414540ddaa801f0e5ccf3'

In [17]:
responses = []
challenge_data = commitments_E[merkle_root_challenge]
merkle_tree = challenge_data['merkle_tree']
for i in challenge_indices:
    challenge_data['commitments'][i]['merkle_proof'] = merkle_tree.get_proof(i)
    responses.append(challenge_data['commitments'][i])

challenge_response = {merkle_root_challenge: responses}
challenge_response

{'b15f3300df9da493f499823cb451ba212de07cafbb6414540ddaa801f0e5ccf3': [{'index': 5,
   'hash': 58759823136492660437955875546910300593624405420820032876480899369228497557688,
   'data_chunk': b'\x96\x0eP\xa2\x9c\xd0t\x14:\xd2\x9d\x85\x81uk\x07\xeb\xbd_Q\xbf\x17<\x02\xe0C\x89\x94\xd2\x05\xcd2&>\x91\x88\xcb0\xf1X\xef/\x9a\x80\xaa\xab\x15\xc3\xc3M\x15\xfd\xa4\xafDJ\xe9E\xbeA\xc5\xc4\xb3\x87\xd1\x08\xd7\t\xa8[1\\}6UG\xed\x80Q]HunF\xae\xa8\x01\x98\x9e\xec',
   'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f6b81210d00>,
   'randomness': 89414787030287347735153962541662876738178162226911838233531622254192223013027,
   'merkle_proof': [{'left': '31383735383838323632343337313031363630343539363136363534363339333833363435313437343634353439303330323632393533373231353036343434323539363332313930313139312c3137353238323738383232343132323837313238393235343533303338393236383038383236323238333339363739303633383230313334393937303438373434303832373331393638353531'},
    {'right': '313739333330333435333839323

In [18]:
merkle_root = list(challenge_response.keys())[0]
commitments = challenge_response[merkle_root]
print("merkle_root:", merkle_root)
commitments

merkle_root: b15f3300df9da493f499823cb451ba212de07cafbb6414540ddaa801f0e5ccf3


[{'index': 5,
  'hash': 58759823136492660437955875546910300593624405420820032876480899369228497557688,
  'data_chunk': b'\x96\x0eP\xa2\x9c\xd0t\x14:\xd2\x9d\x85\x81uk\x07\xeb\xbd_Q\xbf\x17<\x02\xe0C\x89\x94\xd2\x05\xcd2&>\x91\x88\xcb0\xf1X\xef/\x9a\x80\xaa\xab\x15\xc3\xc3M\x15\xfd\xa4\xafDJ\xe9E\xbeA\xc5\xc4\xb3\x87\xd1\x08\xd7\t\xa8[1\\}6UG\xed\x80Q]HunF\xae\xa8\x01\x98\x9e\xec',
  'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f6b81210d00>,
  'randomness': 89414787030287347735153962541662876738178162226911838233531622254192223013027,
  'merkle_proof': [{'left': '31383735383838323632343337313031363630343539363136363534363339333833363435313437343634353439303330323632393533373231353036343434323539363332313930313139312c3137353238323738383232343132323837313238393235343533303338393236383038383236323238333339363739303633383230313334393937303438373434303832373331393638353531'},
   {'right': '313739333330333435333839323838303034343834393334363532383631303536383532323137383531363831373537313333

In [19]:
len(commitments)

1

In [20]:
commitment_i = commitments[0]
commitment_i

{'index': 5,
 'hash': 58759823136492660437955875546910300593624405420820032876480899369228497557688,
 'data_chunk': b'\x96\x0eP\xa2\x9c\xd0t\x14:\xd2\x9d\x85\x81uk\x07\xeb\xbd_Q\xbf\x17<\x02\xe0C\x89\x94\xd2\x05\xcd2&>\x91\x88\xcb0\xf1X\xef/\x9a\x80\xaa\xab\x15\xc3\xc3M\x15\xfd\xa4\xafDJ\xe9E\xbeA\xc5\xc4\xb3\x87\xd1\x08\xd7\t\xa8[1\\}6UG\xed\x80Q]HunF\xae\xa8\x01\x98\x9e\xec',
 'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f6b81210d00>,
 'randomness': 89414787030287347735153962541662876738178162226911838233531622254192223013027,
 'merkle_proof': [{'left': '31383735383838323632343337313031363630343539363136363534363339333833363435313437343634353439303330323632393533373231353036343434323539363332313930313139312c3137353238323738383232343132323837313238393235343533303338393236383038383236323238333339363739303633383230313334393937303438373434303832373331393638353531'},
  {'right': '3137393333303334353338393238383030343438343933343635323836313035363835323231373835313638313735373133333235383

In [21]:
merkle_proof_i = commitments[0]['merkle_proof']
merkle_proof_i

[{'left': '31383735383838323632343337313031363630343539363136363534363339333833363435313437343634353439303330323632393533373231353036343434323539363332313930313139312c3137353238323738383232343132323837313238393235343533303338393236383038383236323238333339363739303633383230313334393937303438373434303832373331393638353531'},
 {'right': '31373933333033343533383932383830303434383439333436353238363130353638353232313738353136383137353731333332353831303433323534393230383830353335373633333234342c3635313833373730393539383232343736323830303531363039393338343834373038323032303931363831303834313932353431313939373937333433353133303733303637353531303836'},
 {'left': 'f1301a836e0bf77765849712787d24d3c76262fb4fe2e27f4b348b5df27e34fa'}]

In [22]:
commitments[0]

{'index': 5,
 'hash': 58759823136492660437955875546910300593624405420820032876480899369228497557688,
 'data_chunk': b'\x96\x0eP\xa2\x9c\xd0t\x14:\xd2\x9d\x85\x81uk\x07\xeb\xbd_Q\xbf\x17<\x02\xe0C\x89\x94\xd2\x05\xcd2&>\x91\x88\xcb0\xf1X\xef/\x9a\x80\xaa\xab\x15\xc3\xc3M\x15\xfd\xa4\xafDJ\xe9E\xbeA\xc5\xc4\xb3\x87\xd1\x08\xd7\t\xa8[1\\}6UG\xed\x80Q]HunF\xae\xa8\x01\x98\x9e\xec',
 'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f6b81210d00>,
 'randomness': 89414787030287347735153962541662876738178162226911838233531622254192223013027,
 'merkle_proof': [{'left': '31383735383838323632343337313031363630343539363136363534363339333833363435313437343634353439303330323632393533373231353036343434323539363332313930313139312c3137353238323738383232343132323837313238393235343533303338393236383038383236323238333339363739303633383230313334393937303438373434303832373331393638353531'},
  {'right': '3137393333303334353338393238383030343438343933343635323836313035363835323231373835313638313735373133333235383

In [23]:
for commitment_i in commitments:
    index = commitment_i['index']
    commitment = commitment_i['point']
    data = commitment_i['data_chunk']
    r = commitment_i['randomness']
    merkle_proof_i = commitment_i['merkle_proof']

    if not committer.open(commitment, hash_data(data), r):
        print(f"Opening commitment {index} failed")

    if not validate_merkle_proof(
        merkle_proof_i, ecc_point_to_hex(commitment_i['point']), merkle_root
    ):
        print(f"Merkle proof {index} validation failed")


Opening: Hashed Value = 58759823136492660437955875546910300593624405420820032876480899369228497557688
Random Value = 89414787030287347735153962541662876738178162226911838233531622254192223013027
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b81210e50>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b81210d00>


In [24]:
def verify(committer, commitments, merkle_root):
    for commitment_i in commitments:
        index = commitment_i['index']
        commitment = commitment_i['point']
        data = commitment_i['data_chunk']
        r = commitment_i['randomness']
        merkle_proof_i = commitment_i['merkle_proof']

        if not committer.open(commitment, hash_data(data), r):
            print(f"Opening commitment {index} failed")
            return False

        if not validate_merkle_proof(
            merkle_proof_i, ecc_point_to_hex(commitment_i['point']), merkle_root
        ):
            print(f"Merkle proof {index} validation failed")
            return False

    return True

In [25]:
verify(committer, commitments, merkle_root)


Opening: Hashed Value = 58759823136492660437955875546910300593624405420820032876480899369228497557688
Random Value = 89414787030287347735153962541662876738178162226911838233531622254192223013027
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b81211cc0>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b81210d00>


True

In [31]:
def recommit_data(committer, challenge_indices, merkle_tree, data):
    new_commitments = {}
    for i in challenge_indices:
        c, m_val, r = committer.commit(data[i])
        new_commitments[i] = {
            "commitment": c,
            "random_vals": r
        }
        merkle_tree.update_leaf(i, ecc_point_to_hex(c))
    new_merkle_root = merkle_tree.get_merkle_root()
    return new_merkle_root, new_commitments

new_merkle_root, new_commitments = recommit_data(committer, challenge_indices, merkle_tree, chunks)
print("new_merkle_root:", new_merkle_root)
new_commitments

Committing: Data = b'\x96\x0eP\xa2\x9c\xd0t\x14:\xd2\x9d\x85\x81uk\x07\xeb\xbd_Q\xbf\x17<\x02\xe0C\x89\x94\xd2\x05\xcd2&>\x91\x88\xcb0\xf1X\xef/\x9a\x80\xaa\xab\x15\xc3\xc3M\x15\xfd\xa4\xafDJ\xe9E\xbeA\xc5\xc4\xb3\x87\xd1\x08\xd7\t\xa8[1\\}6UG\xed\x80Q]HunF\xae\xa8\x01\x98\x9e\xec'
Hashed Value = 58759823136492660437955875546910300593624405420820032876480899369228497557688
Random Value = 39709856185307719836227206246681288794464590277289443003111865814609719469406
Computed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b80f83070>

new_merkle_root: dcb77c6f959032000abbfb616a16d89576851f3fcce2eea9f043d587e5a072d0


{5: {'commitment': <Crypto.PublicKey.ECC.EccPoint at 0x7f6b80f83070>,
  'random_vals': 39709856185307719836227206246681288794464590277289443003111865814609719469406}}

In [34]:
def recommit_data(committer, challenge_indices, merkle_tree, data):
    # new_commitments = {}
    new_commitments = []
    for i in challenge_indices:
        c, m_val, r = committer.commit(data[i])
        commitment_hash = hash_data(ecc_point_to_hex(c)) # Assuming a hash_function is available.
        new_commitments.append( {
            "index": i,
            "hash": commitment_hash,
            "data_chunk": data[i],
            "point": c,
            "randomness": r,
            "merkle_proof": None
        })
        merkle_tree.update_leaf(i, ecc_point_to_hex(c))
    new_merkle_root = merkle_tree.get_merkle_root()
    return new_merkle_root, new_commitments

new_merkle_root, new_commitments = recommit_data(committer, challenge_indices, merkle_tree, chunks)
print("new_merkle_root:", new_merkle_root)
new_commitments

Committing: Data = b'\x96\x0eP\xa2\x9c\xd0t\x14:\xd2\x9d\x85\x81uk\x07\xeb\xbd_Q\xbf\x17<\x02\xe0C\x89\x94\xd2\x05\xcd2&>\x91\x88\xcb0\xf1X\xef/\x9a\x80\xaa\xab\x15\xc3\xc3M\x15\xfd\xa4\xafDJ\xe9E\xbeA\xc5\xc4\xb3\x87\xd1\x08\xd7\t\xa8[1\\}6UG\xed\x80Q]HunF\xae\xa8\x01\x98\x9e\xec'
Hashed Value = 58759823136492660437955875546910300593624405420820032876480899369228497557688
Random Value = 54812417033580816286149853050786078707957688766681092235687675287123461770141
Computed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b80f46aa0>

new_merkle_root: 2036d3eaabf9dd4b94b9bff504f923c53059a354b202e51d25865864ae991021


[{'index': 5,
  'hash': 24937984349728792724423481539765211658033397470603861219919190764465445669713,
  'data_chunk': b'\x96\x0eP\xa2\x9c\xd0t\x14:\xd2\x9d\x85\x81uk\x07\xeb\xbd_Q\xbf\x17<\x02\xe0C\x89\x94\xd2\x05\xcd2&>\x91\x88\xcb0\xf1X\xef/\x9a\x80\xaa\xab\x15\xc3\xc3M\x15\xfd\xa4\xafDJ\xe9E\xbeA\xc5\xc4\xb3\x87\xd1\x08\xd7\t\xa8[1\\}6UG\xed\x80Q]HunF\xae\xa8\x01\x98\x9e\xec',
  'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f6b80f46aa0>,
  'randomness': 54812417033580816286149853050786078707957688766681092235687675287123461770141,
  'merkle_proof': None}]

In [38]:
# Generate merkle proof for updated indices
for nc in new_commitments:
    nc['merkle_proof'] = merkle_tree.get_proof(nc['index'])
nc['merkle_proof']

[{'left': '31383735383838323632343337313031363630343539363136363534363339333833363435313437343634353439303330323632393533373231353036343434323539363332313930313139312c3137353238323738383232343132323837313238393235343533303338393236383038383236323238333339363739303633383230313334393937303438373434303832373331393638353531'},
 {'right': '31373933333033343533383932383830303434383439333436353238363130353638353232313738353136383137353731333332353831303433323534393230383830353335373633333234342c3635313833373730393539383232343736323830303531363039393338343834373038323032303931363831303834313932353431313939373937333433353133303733303637353531303836'},
 {'left': 'f1301a836e0bf77765849712787d24d3c76262fb4fe2e27f4b348b5df27e34fa'}]

In [39]:
# Reverify new commitments to make sure we're solid
verify(committer, new_commitments, new_merkle_root)


Opening: Hashed Value = 58759823136492660437955875546910300593624405420820032876480899369228497557688
Random Value = 54812417033580816286149853050786078707957688766681092235687675287123461770141
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b80f6cd60>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f6b80f46aa0>


True

In [29]:
# NOTE:
# The hash of the full data object should be used for storage location/lookup
# The merkle tree root will keep updating each round so it's not a good idea to use that
# otherwise we will keep moving the data around in the big-ass merkle tree or database (GUN)